**Baseline Model: XGBoost**

1. re-creating the exact training, validation, & test set used to create the graphs.
2. Run XGBoost collect 5 samples from XGBoost
3. save results


In [ ]:
# mounting google colab drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import os
import numpy as np
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from sklearn.metrics import f1_score, recall_score, precision_score

!pip install xgboost
!pip install xgboost ray[tune] ray[default] xgboost_ray

import xgboost as xgb
import ray
from xgboost_ray import RayDMatrix, train, RayParams
from sklearn.metrics import classification_report

# Change directory to location
loc = "/content/drive/MyDrive/KE_GNN/"
os.chdir(loc)
os.getcwd()

test_df = pd.read_csv('Graph storage/test_df.csv'.format(loc))
df_train_t = pd.read_csv('Graph storage/train_df.csv'.format(loc))
df_train_v = pd.read_csv('Graph storage/valid_df.csv'.format(loc))

features = ['DoW', 'minute', 'number_active_cards', 'number_active_accounts', 'Amount','age at time',  # random varaiables
                 'User error_counter',  'User-Merchant error_counter',  'User-Card error_counter',  'User-MCC error_counter', 'Merchant error_counter', # user error counts = a rolling count of pprevious errors
                 'User - previous_error', 'User-Merchant - previous_error', 'User-Card - previous_error', 'User-MCC - previous_error', 'Merchant - previous_error',  # indicator if the previous payments were errors
                 'User CS', 'User CC', 'User CM', 'User CSTD', 'User CM3', 'User CSTD3', 'User CSTD7', 'User CM7', # user relationship
                 'User-Merchant CS', 'User-Merchant CC', 'User-Merchant CM', 'User-Merchant CSTD','User-Merchant CM3', 'User-Merchant CSTD3', 'User-Merchant CSTD7', 'User-Merchant CM7', # user- merchant relationship
                 'User-Card CS', 'User-Card CC', 'User-Card CM', 'User-Card CSTD',  'User-Card CM3', 'User-Card CSTD3', 'User-Card CSTD7', 'User-Card CM7', # user-card relationship
                 'User-MCC CS', 'User-MCC CC', 'User-MCC CM', 'User-MCC CSTD','User-MCC CM3', 'User-MCC CSTD3', 'User-MCC CSTD7', 'User-MCC CM7', # user - mcc relationship
                 'Merchant CS', 'Merchant CC', 'Merchant CM', 'Merchant CSTD', 'Merchant CM3', 'Merchant CSTD3',  'Merchant CSTD7', 'Merchant CM7', # merchant relationship
                 'User_lag_tester_payment_1', 'User_lag_tester_payment_5', 'User_lag_tester_payment_10', 'User_lag_tester_payment_20', # user test lag payments
                 'User-Merchant_lag_tester_payment_1', 'User-Merchant_lag_tester_payment_5', 'User-Merchant_lag_tester_payment_10', 'User-Merchant_lag_tester_payment_20', # user merchant relationship tester payments
                 'User-Card_lag_tester_payment_1', 'User-Card_lag_tester_payment_5', 'User-Card_lag_tester_payment_10', 'User-Card_lag_tester_payment_20', # user cards tester payment
                 'User-MCC_lag_tester_payment_1', 'User-MCC_lag_tester_payment_5', 'User-MCC_lag_tester_payment_10', 'User-MCC_lag_tester_payment_20', # user MCC tester payments
                 'Merchant_lag_tester_payment_1', 'Merchant_lag_tester_payment_5', 'Merchant_lag_tester_payment_10', 'Merchant_lag_tester_payment_20', # merchant tester paymetns
                 'User occurance 1 mins',  'User occurance 10 mins', # user occ in mins
                 'User-Merchant occurance 1 mins',  'User-Merchant occurance 10 mins', # user merchant occurance in mins
                 'User-Card occurance 1 mins', 'User-Card occurance 10 mins', # user card occurance in mins
                 'User-MCC occurance 1 mins', 'User-MCC occurance 10 mins', # user MCC occurance in mins
                 'Merchant occurance 1 mins','Merchant occurance 10 mins',  # merchants in mins
                 'Error - Bad input', 'Error - Insuf bal', 'Error - Tech Glitch' ,
                 'OH1: Chip Transaction', 'OH1: Online Transaction', 'OH1: Swipe Transaction',
                 'FR: Merchant City', 'FR: Merchant State', 'FR: Zip', 'OH4: Ohio', 'OH4: Online', 'OH4: US',
                 'OH4: high_risk', 'OH4: world_non_us', 'OH5: Ohio', 'OH5: US',

                 'Per Capita Income - Zipcode','FR: Zipcode', 'Gender','FICO Score', 'Total Debt',
                 'FR: MCC', 'OH2: Agricultural Services', 'OH2: Contracted Services', 'OH2: Transportation Services',
                 'OH2: Utility Services', 'OH2: Retail Outlet Services', 'OH2: Clothing Stores',
                 'OH2: Miscellaneous Stores', 'OH2: Business Services', 'OH2: Professional Services and Membership Organizations',
                 'OH2: Government Services',]

                 # 'Fraud2'] # remaining

u_features = ['Per Capita Income - Zipcode','FR: Zipcode', 'Gender','FICO Score', 'Total Debt']

m_features = ['Merchant in Counry',  'FR: MCC',
              'OH2: Agricultural Services', 'OH2: Contracted Services', 'OH2: Transportation Services',
                 'OH2: Utility Services', 'OH2: Retail Outlet Services', 'OH2: Clothing Stores',
                 'OH2: Miscellaneous Stores', 'OH2: Business Services', 'OH2: Professional Services and Membership Organizations',
                 'OH2: Government Services' ]
l_features = ['merch_city_index','state_value']


c_features = ['OH2: Amex', 'OH2: Discover', 'OH2: Mastercard', 'OH2: Visa',
              'OH3: Credit', 'OH3: Debit', 'OH3: Debit (Prepaid)','Credit Limit']


total_features = features + u_features + m_features + c_features


print('Features NOT being used: ')
[x for x in df_train_t.columns.tolist() if x not in total_features]



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Features NOT being used: 


['User',
 'Card',
 'Year',
 'Month',
 'Day',
 'Time',
 'Merchant Name',
 'Merchant City',
 'Merchant State',
 'Zip',
 'MCC',
 'Is Fraud?',
 'PK',
 'date_time',
 'DOB',
 'Zipcode',
 'previous_error',
 'user_card',
 'State',
 'Merchant State2',
 'User State2',
 'Fraud2',
 'transaction_pk',
 'User_index',
 'Merchant_index',
 'user_card_index']

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, classification_report
from joblib import Parallel, delayed

def f1_finder(pred, true, max_val=1):
    '''
    Finds the best threshold for maximizing the F1-score of a binary classifier.

    Args:
        pred: Predicted values for the positive class.
        true: True binary labels.
        max_val: The maximum threshold value to consider.

    Returns:
        The threshold that maximizes the F1-score.
    '''
    thresholds = np.linspace(0, max_val, num=200, endpoint=True)

    def compute_f1(threshold):
        binary_pred = (pred > threshold).astype(int)
        return f1_score(true, binary_pred, zero_division=0.0)

    f1_scores = Parallel(n_jobs=-1)(delayed(compute_f1)(x) for x in thresholds)

    best_index = np.argmax(f1_scores)
    best_x = thresholds[best_index]
    return best_x

def xg_boost(run_number, total_features, df_train_t, df_train_v, df_test):
    '''
    Trains and evaluates an XGBoost model for fraud detection.

    Args:
        run_number: An identifier for the current run.
        total_features: A list of features to use for training.
        df_train_t: Training DataFrame containing features and target.
        df_train_v: Validation DataFrame containing features and target.
        df_test: Test DataFrame containing features and target.

    Returns:
        A list containing performance metrics on validation and test sets.
    '''
    total_features = list(set(total_features))  # Replace with the actual list of feature names
    target = 'Is Fraud?'  # Replace with the actual target column name

    dtrain = xgb.DMatrix(df_train_t[total_features], label=df_train_t[target])
    dvalid = xgb.DMatrix(df_train_v[total_features], label=df_train_v[target])
    dtest = xgb.DMatrix(df_test[total_features], label=df_test[target])

    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'max_depth': 6,
        'learning_rate': 0.1,
        'n_estimators': 100
    }

    evals_result = {}
    bst = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=50,
        evals=[(dvalid, 'validation')],
        evals_result=evals_result
    )

    y_pred_prob_valid = bst.predict(dvalid)
    valid_threshold = f1_finder(y_pred_prob_valid, df_train_v[target])
    y_pred_valid = (y_pred_prob_valid > valid_threshold).astype(int)

    valid_f1 = f1_score(df_train_v[target], y_pred_valid, zero_division=0.0)
    valid_recall = recall_score(df_train_v[target], y_pred_valid)
    valid_prc = precision_score(df_train_v[target], y_pred_valid)

    y_pred_prob_test = bst.predict(dtest)

    test_threshold = f1_finder(y_pred_prob_test, df_test[target])

    y_pred_test_opt = (y_pred_prob_test > test_threshold).astype(int)
    y_pred_test_val = (y_pred_prob_test > valid_threshold).astype(int)

    test_f1_v = f1_score(df_test[target], y_pred_test_val, zero_division=0.0)
    test_recall_v = recall_score(df_test[target], y_pred_test_val)
    test_prc_v = precision_score(df_test[target], y_pred_test_val)

    test_f1_opt = f1_score(df_test[target], y_pred_test_opt, zero_division=0.0)
    test_recall_opt = recall_score(df_test[target], y_pred_test_opt)
    test_prc_opt = precision_score(df_test[target], y_pred_test_opt)

    print('Validation optimised results:')
    print(classification_report(df_test[target], y_pred_test_val))

    print('Test optimised result:')
    print(classification_report(df_test[target], y_pred_test_opt))

    return [
        run_number,
        valid_threshold, valid_f1, valid_recall, valid_prc,
        valid_threshold, test_f1_v, test_recall_v, test_prc_v,
        test_threshold, test_f1_opt, test_recall_opt, test_prc_opt
    ]

results = []
for x in range(5):
    results.append(xg_boost(x, total_features, df_train_t, df_train_v, test_df))

df = pd.DataFrame(results, columns=['run number',
                                    'valid_threshold', 'Valid f1','valid recall','valid pres',
                                    'valid_threshold', 'test f1 v_thres', 'test recall v_thre', 'test prec v_thres',
                                    'test_threshold', 'test f1 t_thres', 'test recall t_thre', 'test prec t_thres',

                                    ])
df.to_csv('{}Main Experiment/output/XGBOOST/XG_boost_results.csv'.format(loc))


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:44:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-logloss:0.11720
[1]	validation-logloss:0.10576
[2]	validation-logloss:0.09555
[3]	validation-logloss:0.08625
[4]	validation-logloss:0.07784
[5]	validation-logloss:0.07054
[6]	validation-logloss:0.06402
[7]	validation-logloss:0.05817
[8]	validation-logloss:0.05279
[9]	validation-logloss:0.04790
[10]	validation-logloss:0.04341
[11]	validation-logloss:0.03944
[12]	validation-logloss:0.03587
[13]	validation-logloss:0.03261
[14]	validation-logloss:0.02969
[15]	validation-logloss:0.02707
[16]	validation-logloss:0.02471
[17]	validation-logloss:0.02255
[18]	validation-logloss:0.02062
[19]	validation-logloss:0.01887
[20]	validation-logloss:0.01730
[21]	validation-logloss:0.01585
[22]	validation-logloss:0.01453
[23]	validation-logloss:0.01336
[24]	validation-logloss:0.01230
[25]	validation-logloss:0.01134
[26]	validation-logloss:0.01048
[27]	validation-logloss:0.00970
[28]	validation-logloss:0.00898
[29]	validation-logloss:0.00832
[30]	validation-logloss:0.00774
[31]	validation-lo

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:48:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-logloss:0.11720
[1]	validation-logloss:0.10576
[2]	validation-logloss:0.09555
[3]	validation-logloss:0.08625
[4]	validation-logloss:0.07784
[5]	validation-logloss:0.07054
[6]	validation-logloss:0.06402
[7]	validation-logloss:0.05817
[8]	validation-logloss:0.05279
[9]	validation-logloss:0.04790
[10]	validation-logloss:0.04341
[11]	validation-logloss:0.03944
[12]	validation-logloss:0.03587
[13]	validation-logloss:0.03261
[14]	validation-logloss:0.02969
[15]	validation-logloss:0.02707
[16]	validation-logloss:0.02471
[17]	validation-logloss:0.02255
[18]	validation-logloss:0.02062
[19]	validation-logloss:0.01887
[20]	validation-logloss:0.01730
[21]	validation-logloss:0.01585
[22]	validation-logloss:0.01453
[23]	validation-logloss:0.01336
[24]	validation-logloss:0.01230
[25]	validation-logloss:0.01134
[26]	validation-logloss:0.01048
[27]	validation-logloss:0.00970
[28]	validation-logloss:0.00898
[29]	validation-logloss:0.00832
[30]	validation-logloss:0.00774
[31]	validation-lo

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:51:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-logloss:0.11720
[1]	validation-logloss:0.10576
[2]	validation-logloss:0.09555
[3]	validation-logloss:0.08625
[4]	validation-logloss:0.07784
[5]	validation-logloss:0.07054
[6]	validation-logloss:0.06402
[7]	validation-logloss:0.05817
[8]	validation-logloss:0.05279
[9]	validation-logloss:0.04790
[10]	validation-logloss:0.04341
[11]	validation-logloss:0.03944
[12]	validation-logloss:0.03587
[13]	validation-logloss:0.03261
[14]	validation-logloss:0.02969
[15]	validation-logloss:0.02707
[16]	validation-logloss:0.02471
[17]	validation-logloss:0.02255
[18]	validation-logloss:0.02062
[19]	validation-logloss:0.01887
[20]	validation-logloss:0.01730
[21]	validation-logloss:0.01585
[22]	validation-logloss:0.01453
[23]	validation-logloss:0.01336
[24]	validation-logloss:0.01230
[25]	validation-logloss:0.01134
[26]	validation-logloss:0.01048
[27]	validation-logloss:0.00970
[28]	validation-logloss:0.00898
[29]	validation-logloss:0.00832
[30]	validation-logloss:0.00774
[31]	validation-lo

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:55:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-logloss:0.11720
[1]	validation-logloss:0.10576
[2]	validation-logloss:0.09555
[3]	validation-logloss:0.08625
[4]	validation-logloss:0.07784
[5]	validation-logloss:0.07054
[6]	validation-logloss:0.06402
[7]	validation-logloss:0.05817
[8]	validation-logloss:0.05279
[9]	validation-logloss:0.04790
[10]	validation-logloss:0.04341
[11]	validation-logloss:0.03944
[12]	validation-logloss:0.03587
[13]	validation-logloss:0.03261
[14]	validation-logloss:0.02969
[15]	validation-logloss:0.02707
[16]	validation-logloss:0.02471
[17]	validation-logloss:0.02255
[18]	validation-logloss:0.02062
[19]	validation-logloss:0.01887
[20]	validation-logloss:0.01730
[21]	validation-logloss:0.01585
[22]	validation-logloss:0.01453
[23]	validation-logloss:0.01336
[24]	validation-logloss:0.01230
[25]	validation-logloss:0.01134
[26]	validation-logloss:0.01048
[27]	validation-logloss:0.00970
[28]	validation-logloss:0.00898
[29]	validation-logloss:0.00832
[30]	validation-logloss:0.00774
[31]	validation-lo

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:58:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-logloss:0.11720
[1]	validation-logloss:0.10576
[2]	validation-logloss:0.09555
[3]	validation-logloss:0.08625
[4]	validation-logloss:0.07784
[5]	validation-logloss:0.07054
[6]	validation-logloss:0.06402
[7]	validation-logloss:0.05817
[8]	validation-logloss:0.05279
[9]	validation-logloss:0.04790
[10]	validation-logloss:0.04341
[11]	validation-logloss:0.03944
[12]	validation-logloss:0.03587
[13]	validation-logloss:0.03261
[14]	validation-logloss:0.02969
[15]	validation-logloss:0.02707
[16]	validation-logloss:0.02471
[17]	validation-logloss:0.02255
[18]	validation-logloss:0.02062
[19]	validation-logloss:0.01887
[20]	validation-logloss:0.01730
[21]	validation-logloss:0.01585
[22]	validation-logloss:0.01453
[23]	validation-logloss:0.01336
[24]	validation-logloss:0.01230
[25]	validation-logloss:0.01134
[26]	validation-logloss:0.01048
[27]	validation-logloss:0.00970
[28]	validation-logloss:0.00898
[29]	validation-logloss:0.00832
[30]	validation-logloss:0.00774
[31]	validation-lo